### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB
>1. Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

        *Наименование вакансии
        *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
        *Ссылку на саму вакансию        
        *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

>2. Задание похожее, но подход к сбору данных здесь другой.
Необходимо собрать информацию по продуктам питания с сайтов: https://rskrf.ru/ratings/produkty-pitaniya/ и https://roscontrol.com/category/produkti/ Получившийся список должен содержать:

        *Наименование продукта
        *Категорию продукта        (например Бакалея)
        *Подкатегорию продукта    (апример "Рис круглозерный")
        *Параметр "безопасность"
        *Параметр "качество"
        *Общий балл
        *Сайт, откуда получена информация
Данная структура должна быть одинаковая для продуктов с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

Можно выполнить по своему желанию одно любое задание или оба при желании и возможности.

In [ ]:
#pip install selenium

In [262]:
%config IPCompleter.greedy=True
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
vacancy_name = input('Введите вакансию:')

Введите вакансию:dba


* User-agent & general link

In [317]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'}
link = 'https://spb.hh.ru/search/vacancy'

In [343]:
def get_pages():
    columns = ['VacancyName','Link', 'Min Rate', 'Max Rate']
    df = pd.DataFrame(columns=columns)
    r = requests.get(f'{link}??L_is_autosearch=false&clusters=true&enable_snippets=true&text=\
    {vacancy_name}&page=0',headers = headers)
    soup = bs(r.text,'html.parser')
    max_pages = int(max([a['data-page'] for a in soup.find_all('a',{'class': 'bloko-button HH-Pager-Control'})]))
    if max_pages > 1:
        for i in range(max_pages):
            r = requests.get(f'{link}??L_is_autosearch=false&clusters=true&enable_snippets=true&text=\
            {vacancy_name}&page={i}',headers = headers)
            soup = bs(r.text,'html.parser')
            for j in range(len(soup.find_all('a',{'class': 'bloko-link HH-LinkModifier'}))):
                
                name = soup.find_all('a',{'class': 'bloko-link HH-LinkModifier'})[j].string
                
                link2 = [a['href'] for a in soup.find_all('a',{'class': 'bloko-link HH-LinkModifier'}, href=True)][j]\
                .replace(f'?query={vacancy_name}','')
                
                all_tags = soup.find_all('div',{'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})[j]
                
                sal = all_tags.find_all('div',{'class': 'vacancy-serp-item__compensation'})
                
                if len(all_tags.find_all('div',{'class': 'vacancy-serp-item__compensation'})) == 0:
                    min_rate = 'Зарплата не указана'
                    max_rate = ''
                if (len(sal) == 1) and (len(re.findall(r'[-]',sal[0].string)) == 0):
                    if sal[0].string.replace(u'\xa0', '').split(' ', 1)[0] == 'от':
                        min_rate = sal[0].string.replace(u'\xa0', '').split(' ', 1)[1]
                        max_rate = ''
                    else:
                        min_rate = ''
                        max_rate = sal[0].string.replace(u'\xa0', '').split(' ', 1)[1]
                if (len(sal) == 1) and (len(re.findall(r'[-]',sal[0].string)) == 1):
                    min_rate = sal[0].string.replace(u'\xa0', '').split('-', 1)[0]
                    max_rate = sal[0].string.replace(u'\xa0', '').split('-', 1)[1]
                df=df.append([{'VacancyName':name,'Link':link2,'Min Rate':min_rate,'Max Rate':max_rate}], ignore_index = True)
     
    return df[['VacancyName','Min Rate','Max Rate','Link']];

In [344]:
get_pages()

,VacancyName,Min Rate,Max Rate,Link
0,PL/pgSQL Developer / PostgreSQL DBA,160000 руб.,,https://spb.hh.ru/vacancy/30996704
1,DBA,Зарплата не указана,,https://spb.hh.ru/vacancy/33518837
2,Администратор баз данных (Oracle DBA)/Разработчик,2000 бел.руб.,,https://spb.hh.ru/vacancy/33150417
3,Администратор баз данных (DBA),Зарплата не указана,,https://spb.hh.ru/vacancy/32546564
4,Администратор баз данных (DBA),Зарплата не указана,,https://spb.hh.ru/vacancy/30423387
5,Oracle DBA,Зарплата не указана,,https://spb.hh.ru/vacancy/33175516
6,Администратор баз данных SQL (DBA),Зарплата не указана,,https://spb.hh.ru/vacancy/32938354
7,Администратор баз данных MS SQL (Разработчик),Зарплата не указана,,https://spb.hh.ru/vacancy/32376002
8,MS SQL DBA,Зарплата не указана,,https://spb.hh.ru/vacancy/31822344
9,Администратор баз данных Oracle/ Oracle DBA,Зарплата не указана,,https://spb.hh.ru/vacancy/32972828
